# Imports

In [72]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/daniildiveev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Data loading

In [2]:
data = pd.read_csv("../data/kp_films.csv")

In [3]:
data

,kp_id,rating_kp,genres,description
0,12819,7.70,['триллер' 'мелодрама' 'криминал'],"Миллиардер Томас Краун, пресыщенный финансист,..."
1,1260096,-1.00,['семейный' 'для' 'кино' 'ребенок'],Банда саранчи во главе с безжалостным Боссом о...
2,894673,4.40,['вестерн' 'иностранный' 'триллер' 'мистика' '...,"В горах живёт обособленная община, возглавляем..."
3,485364,6.06,['семейный' 'для' 'кино' 'короткометражный' 'р...,"Фильм высмеивает тех, кто понимает трудовую ди..."
4,1289632,6.40,['семейный' 'приключение' 'кино'],"В скучном городке Линдсборо всё меняется, когд..."
...,...,...,...,...
18636,806159,5.31,['семейный' 'аниме' 'приключение' 'фантастика'...,"Сатоши готовится к новому турниру, где парню н..."
18637,625381,6.40,['русский' 'мультфильм' 'для' 'ребенок' 'детск...,На нелепом и забавном мужчине по прозвищу Пара...
18638,4535755,-1.00,['семейный' 'мультфильм' 'иностранный' 'для' '...,Очаровательная сказка о маленьком лягушонке с ...
18639,1111361,6.60,['драма'],Криминальная драма о реальной истории жизни си...


# Data preprocessing

In [10]:
vectorizer = TfidfVectorizer()

In [11]:
corpus = vectorizer.fit_transform(data.description)

In [15]:
corpus

<18641x133400 sparse matrix of type '<class 'numpy.float64'>'
	with 1371699 stored elements in Compressed Sparse Row format>

In [28]:
genres = data.genres.to_list()

for i in range(len(genres)):
    genres[i] = genres[i].replace('[', '')
    genres[i] = genres[i].replace(']', '')
    genres[i] = genres[i].replace(',', '')
    genres[i] = genres[i].replace("'", '')
    genres[i] = genres[i].split()

In [35]:
genres_corpus = []

for genres_list in genres:
    genres_corpus += genres_list

genres_corpus = set(genres_corpus)

In [62]:
genres_corpus = genres_corpus.difference(stopwords.words("russian")) #removing stopwords from genres

In [63]:
for genre in genres_corpus:
    data[genre] = [1 if genre in genres[i] else 0 for i in range(data.shape[0])]

In [64]:
data

,kp_id,rating_kp,genres,description,короткометражный,боевик,взрослый,песня,телешоу,нуар,...,передача,драма,молодежный,биография,знать,военный,кино,медицинский,все,свет
0,12819,7.70,['триллер' 'мелодрама' 'криминал'],"Миллиардер Томас Краун, пресыщенный финансист,...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1260096,-1.00,['семейный' 'для' 'кино' 'ребенок'],Банда саранчи во главе с безжалостным Боссом о...,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,894673,4.40,['вестерн' 'иностранный' 'триллер' 'мистика' '...,"В горах живёт обособленная община, возглавляем...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,485364,6.06,['семейный' 'для' 'кино' 'короткометражный' 'р...,"Фильм высмеивает тех, кто понимает трудовую ди...",1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,1289632,6.40,['семейный' 'приключение' 'кино'],"В скучном городке Линдсборо всё меняется, когд...",0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18636,806159,5.31,['семейный' 'аниме' 'приключение' 'фантастика'...,"Сатоши готовится к новому турниру, где парню н...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18637,625381,6.40,['русский' 'мультфильм' 'для' 'ребенок' 'детск...,На нелепом и забавном мужчине по прозвищу Пара...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18638,4535755,-1.00,['семейный' 'мультфильм' 'иностранный' 'для' '...,Очаровательная сказка о маленьком лягушонке с ...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18639,1111361,6.60,['драма'],Криминальная драма о реальной истории жизни си...,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [70]:
genre_to_num = {genre : (data[genre] != 0).sum() for genre in genres_corpus}
genre_to_num = sorted(genre_to_num.items(), key=lambda x: x[1], reverse=True)

In [73]:
genre_to_num = np.array(genre_to_num)

In [76]:
genre_to_num[:, 1].astype(np.int32)

array([7122, 4522, 4067, 3698, 3654, 2937, 2782, 2672, 2364, 2213, 2203,
       2079, 1989, 1792, 1699, 1434, 1250, 1223, 1190, 1180, 1092,  966,
        787,  707,  607,  572,  372,  281,  277,  264,  259,  254,  252,
        206,  183,  182,  130,  122,   97,   94,   91,   87,   75,   75,
         68,   65,   65,   57,   57,   57,   56,   54,   48,   47,   45,
         45,   40,   34,   32,   31,   30,   23,   23,   22,   21,   17,
         15,   12,   12,   12,   12,    8,    7,    6,    5,    5,    4,
          4,    4,    3,    3,    3,    3,    3,    2,    2,    2,    1,
          1,    1,    1,    1,    1,    1], dtype=int32)